In [19]:
# Install all the libraries needed for the whole project
!pip install pandas beautifulsoup4 textstat scikit-learn sentence-transformers nltk

In [20]:

# SEO Content Quality & Duplicate Detection - HTML Parsing Step


import os
import re
import pandas as pd
from bs4 import BeautifulSoup

#1. Define paths

input_path = "data/data.csv"
output_path = "data/extracted_content.csv"

# Make sure the 'data' directory exists
os.makedirs("data", exist_ok=True)

# === 2. Load dataset ========================================================
print(f"Reading dataset from: {input_path}\n")

# Try to load the file, give a clear error if it's missing
try:
    df = pd.read_csv(input_path)
except FileNotFoundError:
    print(f"--- ERROR! ---")
    print(f"Could not find {input_path}.")
    print("Please create a 'data' folder in the Colab file browser and upload your 'data.csv' file.")
    # Stop the script if the file isn't there
    raise

print(f"Loaded {len(df)} records.")
print(f"Columns: {df.columns.tolist()}")

# === 3. HTML parsing function ==============================================

def parse_html(html_string):
    """
    Parse HTML content to extract title, body text, and word count.
    """
    try:
        # Not all 'html_content' cells might be valid strings, check for NaNs
        if not isinstance(html_string, str):
            return {"title": None, "body_text": None, "word_count": 0}

        soup = BeautifulSoup(html_string, "html.parser")

        # Extract <title> tag text
        title_tag = soup.find("title")
        title = title_tag.get_text(strip=True) if title_tag else None

        # Extract main content: Try <article>, then <main>, then all <p> tags
        # This fallback logic is key for handling different site structures.
        body_text = ""
        if soup.find("article"):
            body_text = soup.find("article").get_text(separator=" ", strip=True)
        elif soup.find("main"):
            body_text = soup.find("main").get_text(separator=" ", strip=True)
        else:
            # Fallback: just stitch all paragraphs together
            paragraphs = soup.find_all("p")
            body_text = " ".join(p.get_text(strip=True) for p in paragraphs)

        # Clean up extra whitespace (newlines, tabs, multiple spaces)
        body_text = re.sub(r"\s+", " ", body_text).strip()

        # Word count
        word_count = len(body_text.split()) if body_text else 0

        return {"title": title, "body_text": body_text, "word_count": word_count}

    except Exception as e:
        # Catch any other weird parsing errors
        print(f"⚠️ Error parsing HTML: {str(e)[:100]}")
        return {"title": None, "body_text": None, "word_count": 0}

# === 4. Apply parsing ======================================================

print("\n🔍 Parsing HTML content... (this might take a minute)")

# Run the function on every row in the 'html_content' column
parsed_results = df["html_content"].apply(parse_html)

# This is a clean way to turn a Series of dicts into a DataFrame
parsed_df = parsed_results.apply(pd.Series)

# Combine our new columns (title, etc.) with the original 'url'
extracted_df = pd.concat([df["url"], parsed_df], axis=1)

# === 5. Stats & Save =======================================================

print("\n📊 Parsing Statistics:")
# Filter out any rows where we failed to get text
valid_rows = extracted_df["body_text"].notna() & (extracted_df["word_count"] > 0)
num_valid = valid_rows.sum()

print(f"Successfully parsed: {num_valid} / {len(extracted_df)}")
print(f"Average word count (of valid pages): {extracted_df.loc[valid_rows, 'word_count'].mean():.1f}")
print(f"Pages with <500 words (thin content): {(extracted_df['word_count'] < 500).sum()}")

# Drop failed/empty rows before saving
extracted_df = extracted_df[valid_rows].copy()

# Save to CSV. Using utf-8-sig avoids potential encoding issues in Excel
extracted_df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\n Extracted content saved to: {output_path}")

# Preview
print("\n Sample of extracted content:")
print(extracted_df.head(3))

Reading dataset from: data/data.csv

Loaded 81 records.
Columns: ['url', 'html_content']

🔍 Parsing HTML content... (this might take a minute)

📊 Parsing Statistics:
Successfully parsed: 68 / 81
Average word count (of valid pages): 3290.9
Pages with <500 words (thin content): 36

 Extracted content saved to: data/extracted_content.csv

 Sample of extracted content:
                                                 url  \
0     https://www.cm-alliance.com/cybersecurity-blog   
1    https://www.varonis.com/blog/cybersecurity-tips   
2  https://www.cisecurity.org/insights/blog/11-cy...   

                                               title  \
0                                Cyber Security Blog   
1  Top 10 Cybersecurity Awareness Tips: How to St...   
2  11 Cyber Defense Tips to Stay Secure at Work a...   

                                           body_text  word_count  
0  Cyber Crisis Tabletop Exercise Cyber Security ...       325.0  
1  Cybersecurity is gaining more importance glob

In [21]:
# ------------------------------------------------------------
# Phase 2: Text Preprocessing & Feature Engineering
# ------------------------------------------------------------

import nltk
import textstat
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

# === 1. Download NLTK data ==================================================
# We need the 'punkt' tokenizer for splitting text into sentences
nltk.download('punkt')
# We also need stopwords to filter them out for keyword extraction
nltk.download('stopwords')

# === 2. Load extracted data ================================================
print("Loading extracted_content.csv...")
features_df = pd.read_csv("data/extracted_content.csv")

# === 3. Define Feature Engineering Functions ==============================

def get_sentence_count(text):
    """Calculates the total number of sentences in a text."""
    try:
        return len(nltk.sent_tokenize(text))
    except:
        return 0

def get_readability_score(text):
    """Calculates the Flesch Reading Ease score."""
    try:
        return textstat.flesch_reading_ease(text)
    except:
        return 0

# Let's get our text corpus ready
corpus = features_df['body_text'].tolist()

# === 4. Calculate Basic & Readability Features ==========================
print("Calculating basic text features (sentences, readability)...")

features_df['sentence_count'] = features_df['body_text'].apply(get_sentence_count)
features_df['flesch_reading_ease'] = features_df['body_text'].apply(get_readability_score)

# === 5. Extract Keywords (TF-IDF) =======================================
print("Extracting keywords using TF-IDF...")

# We'll use TF-IDF to find the top keywords.
# min_df=2: Ignore words that appear in only 1 document
# max_df=0.8: Ignore words that appear in > 80% of documents (too common)
tfidf_vec = TfidfVectorizer(stop_words='english', max_features=1000, min_df=2, max_df=0.8)
tfidf_matrix = tfidf_vec.fit_transform(corpus)

# Get the actual feature names (the words)
feature_names = tfidf_vec.get_feature_names_out()

def get_top_keywords(tfidf_row, feature_names, top_n=5):
    """Gets the top_n keywords from a single document's TF-IDF vector."""
    # Get the indices of the top_n scores
    top_indices = tfidf_row.argsort()[-top_n:][::-1]
    # Get the words and scores
    keywords = [feature_names[i] for i in top_indices]
    return ', '.join(keywords) # Store as a simple comma-separated string

# Get the dense version of the row from the sparse matrix
top_keywords_list = [get_top_keywords(tfidf_matrix[i].toarray().flatten(), feature_names) for i in range(len(corpus))]
features_df['top_keywords'] = top_keywords_list

# === 6. Generate Sentence Embeddings ====================================
print("Generating sentence embeddings... (This may take a moment)")

# 'all-MiniLM-L6-v2' is a fantastic model: it's fast, small, and great
# for tasks like similarity and clustering.
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# This encodes all 68 text bodies into a 384-dimension vector
# We do this in one batch, which is much faster.
embeddings = embedding_model.encode(corpus, show_progress_bar=True)

print(f"Embeddings generated with shape: {embeddings.shape}")

# === 7. Save Features & Embeddings ======================================

# We'll save the text features to 'features.csv'
# It's bad practice to save a giant array (the embedding) in a CSV cell.
# The professional way is to save the embeddings in a separate, optimized file.
# We'll use NumPy's .npy format for this.

# Let's clean up the df before saving
features_to_save = features_df[['url', 'word_count', 'sentence_count', 'flesch_reading_ease', 'top_keywords']]

features_to_save.to_csv('data/features.csv', index=False, encoding='utf-8-sig')
print(f"\n Text features saved to data/features.csv")

# Save the embeddings array
np.save('data/embeddings.npy', embeddings)
print(f" Embeddings saved to data/embeddings.npy")

# Preview
print("\n🔹 Sample of features:")
print(features_to_save.head(3))

Loading extracted_content.csv...
Calculating basic text features (sentences, readability)...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Extracting keywords using TF-IDF...
Generating sentence embeddings... (This may take a moment)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Embeddings generated with shape: (68, 384)

 Text features saved to data/features.csv
 Embeddings saved to data/embeddings.npy

🔹 Sample of features:
                                                 url  word_count  \
0     https://www.cm-alliance.com/cybersecurity-blog       325.0   
1    https://www.varonis.com/blog/cybersecurity-tips      1700.0   
2  https://www.cisecurity.org/insights/blog/11-cy...      1058.0   

   sentence_count  flesch_reading_ease  \
0               5            -6.967854   
1              92            41.465000   
2              73            53.262918   

                                        top_keywords  
0  cyber, cybersecurity, training, management, se...  
1                   data, access, security, app, mfa  
2          authentication, don, cyber, home, protect  


In [22]:

# Phase 3: Duplicate Content Detection

from sklearn.metrics.pairwise import cosine_similarity

# === 1. Load Features and Embeddings =====================================
print("Loading features and embeddings...")
try:
    features_df = pd.read_csv('data/features.csv')
    embeddings = np.load('data/embeddings.npy')
    urls = features_df['url'].tolist() # Get a list of URLs for reference

    # Check that our data matches
    if len(features_df) != embeddings.shape[0]:
        print(f"--- ERROR! ---")
        print(f"Mismatch! features.csv has {len(features_df)} rows but embeddings.npy has {embeddings.shape[0]} rows.")
        raise

except FileNotFoundError:
    print("--- ERROR! ---")
    print("Could not find features.csv or embeddings.npy. Did Phase 2 run correctly?")
    raise

# === 2. Calculate Cosine Similarity ======================================
print("Calculating cosine similarity matrix...")
# This computes the similarity score between every document and every other document
# The result is a 68x68 matrix (in your case)
cosine_sim_matrix = cosine_similarity(embeddings)

# === 3. Find Duplicate Pairs =============================================
print("Finding duplicate pairs...")
# Per the assignment, we'll use a threshold of 0.80
SIMILARITY_THRESHOLD = 0.80
duplicates_found = []

# We iterate through the *upper triangle* of the matrix
# This stops us from checking a doc against itself (i==j)
# and from checking pairs twice (e.g., A-B and B-A)
for i in range(len(cosine_sim_matrix)):
    for j in range(i + 1, len(cosine_sim_matrix)):
        similarity = cosine_sim_matrix[i][j]

        if similarity > SIMILARITY_THRESHOLD:
            # We found a pair!
            duplicates_found.append({
                'url_a': urls[i],
                'url_b': urls[j],
                'similarity_score': similarity
            })

# === 4. Save Duplicates ==================================================
df_duplicates = pd.DataFrame(duplicates_found)
df_duplicates.to_csv('data/duplicates.csv', index=False, encoding='utf-8-sig')
print(f" Found {len(df_duplicates)} duplicate pairs. Saved to data/duplicates.csv")

# === 5. Print Final Analysis Summary =====================================
# This part answers the prompt's analysis questions directly
thin_content_count = (features_df['word_count'] < 500).sum()
total_pages = len(features_df)

print("\n--- SEO Content Analysis Summary ---")
print(f"Total Pages Analyzed: {total_pages}")
print(f"Duplicate Pairs Found (>{SIMILARITY_THRESHOLD*100}% similarity): {len(df_duplicates)}")
print(f"Thin Content Pages (<500 words): {thin_content_count} ({(thin_content_count/total_pages)*100:.1f}%)")

if len(df_duplicates) > 0:
    print("\n🔹 Sample of duplicate pairs:")
    print(df_duplicates.head())

Loading features and embeddings...
Calculating cosine similarity matrix...
Finding duplicate pairs...
 Found 10 duplicate pairs. Saved to data/duplicates.csv

--- SEO Content Analysis Summary ---
Total Pages Analyzed: 68
Duplicate Pairs Found (>80.0% similarity): 10
Thin Content Pages (<500 words): 23 (33.8%)

🔹 Sample of duplicate pairs:
                                               url_a  \
0  https://en.wikipedia.org/wiki/Search_engine_op...   
1  https://en.wikipedia.org/wiki/Search_engine_op...   
2  https://en.wikipedia.org/wiki/Search_engine_op...   
3  https://simple.wikipedia.org/wiki/Search_engin...   
4  https://simple.wikipedia.org/wiki/Search_engin...   

                                               url_b  similarity_score  
0  https://simple.wikipedia.org/wiki/Search_engin...          0.995893  
1     https://en.wikipedia.org/wiki/Machine_learning          0.805831  
2  https://simple.wikipedia.org/wiki/Machine_lear...          0.808370  
3     https://en.wikipedia.org

In [23]:

# Phase 4: Content Quality Model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import joblib # For saving our trained model

# === 1. Load Features ======================================================
print("Loading features.csv...")
model_df = pd.read_csv('data/features.csv')

# === 2. Create Quality Labels (Target Variable) ==========================
# We're applying the business rules from the assignment doc
# to create our 'y' variable (the thing we want to predict).

# Define the conditions
conditions = [
    # High Quality
    (model_df['word_count'] > 1500) & (model_df['flesch_reading_ease'] >= 50) & (model_df['flesch_reading_ease'] <= 70),
    # Low Quality
    (model_df['word_count'] < 500) | (model_df['flesch_reading_ease'] < 30)
]

# Define the corresponding labels
labels = ['High', 'Low']

# np.select is a clean way to do this. 'Medium' is the default for anything
# that doesn't meet the High or Low criteria.
model_df['quality_label'] = np.select(conditions, labels, default='Medium')

print("Generated quality labels:")
print(model_df['quality_label'].value_counts())

# === 3. Prepare Data for Modeling ========================================

# Define our features (X) and our target (y)
# We'll use these features to PREDICT the quality_label
feature_columns = ['word_count', 'sentence_count', 'flesch_reading_ease']
X = model_df[feature_columns]
y = model_df['quality_label']

# Split our data: 70% for training, 30% for testing
# random_state=42 ensures we get the same split every time we run
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"\nData split: {len(X_train)} training samples, {len(X_test)} testing samples.")

# ---
# NOTE: With only 68 total samples, our test set is tiny (~21 samples).
# Model performance might not be stable, but we follow the process.
# ---

# === 4. Train a Model ====================================================
print("Training Logistic Regression model...")

# We'll use LogisticRegression. It's simple, fast, and easy to interpret.
# `class_weight='balanced'` helps the model handle cases where we have
# way more 'Low' than 'High' labels.
model = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)
model.fit(X_train, y_train)

print(" Model trained.")

# === 5. Evaluate the Model ===============================================
print("Evaluating model performance...")

# Get predictions on the unseen test data
y_pred = model.predict(X_test)

# Print the results!
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, zero_division=0))

print("\n--- Confusion Matrix ---")
print("Rows = Actual, Columns = Predicted")
print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                 index=model.classes_ + " (Actual)",
                 columns=model.classes_ + " (Predicted)"))

# === 6. Save the Model ===================================================
# We save the trained model so we can use it later (in our demo)
# without having to retrain it every time.
os.makedirs('models', exist_ok=True)
model_path = 'models/quality_model.joblib'
joblib.dump(model, model_path)

print(f"\n Model saved to {model_path}")

Loading features.csv...
Generated quality labels:
quality_label
Low       35
Medium    25
High       8
Name: count, dtype: int64

Data split: 47 training samples, 21 testing samples.
Training Logistic Regression model...
 Model trained.
Evaluating model performance...

--- Classification Report ---
              precision    recall  f1-score   support

        High       0.00      0.00      0.00         2
         Low       0.82      0.82      0.82        11
      Medium       0.43      0.38      0.40         8

    accuracy                           0.57        21
   macro avg       0.42      0.40      0.41        21
weighted avg       0.59      0.57      0.58        21


--- Confusion Matrix ---
Rows = Actual, Columns = Predicted
                 High (Predicted)  Low (Predicted)  Medium (Predicted)
High (Actual)                   0                0                   2
Low (Actual)                    0                9                   2
Medium (Actual)                 3            

In [24]:

# Phase 5: Real-Time Analysis Function (Demo)

import requests
import json

# We need to load our saved model
try:
    demo_model = joblib.load('models/quality_model.joblib')
    print("Loaded trained model 'models/quality_model.joblib'")
except FileNotFoundError:
    print("--- ERROR! ---")
    print("Model file not found. Did Phase 4 run correctly?")
    raise

# We also need the parsing function from Phase 1.
# We can just re-run the cell from Phase 1, or copy it here.
# For a clean notebook, just ensure Phase 1's cell has been run.
# (We'll assume 'parse_html' is already in memory)

def analyze_url(url):
    """
    Takes a single URL, scrapes it, analyzes it, and returns
    a quality report.
    """
    print(f"\n--- Analyzing {url} ---")

    # 1. Fetch HTML
    try:
        # We MUST provide a User-Agent, or many sites will block us (403 Error)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code != 200:
            return {"error": f"Failed to fetch. Status code: {response.status_code}"}

        html_content = response.text

    except Exception as e:
        return {"error": f"Failed to fetch URL: {str(e)}"}

    # 2. Parse HTML (using our function from Phase 1)
    # The 'parse_html' function must be defined in a previous cell!
    try:
        parsed_data = parse_html(html_content)
    except NameError:
        return {"error": "The 'parse_html' function is not defined. Please run the Phase 1 cell."}
    except Exception as e:
        return {"error": f"HTML parsing failed: {str(e)}"}

    title = parsed_data['title']
    body_text = parsed_data['body_text']
    word_count = parsed_data['word_count']

    if word_count == 0:
        return {"error": "Could not extract any body text from the page."}

    # 3. Engineer Features
    sentence_count = get_sentence_count(body_text) # (from Phase 2)
    flesch_reading_ease = get_readability_score(body_text) # (from Phase 2)

    # 4. Make Prediction
    # We create a small DataFrame with the *exact* feature names
    # the model was trained on. This stops the UserWarning.
    feature_names = ['word_count', 'sentence_count', 'flesch_reading_ease']
    features_df_live = pd.DataFrame(
        [[word_count, sentence_count, flesch_reading_ease]],
        columns=feature_names
    )

    quality_label = demo_model.predict(features_df_live)[0]
    quality_probs = demo_model.predict_proba(features_df_live)[0]

    # Create a nice dictionary of the probabilities
    probabilities = {label: f"{prob*100:.1f}%" for label, prob in zip(demo_model.classes_, quality_probs)}

    # 5. Format Output (as per assignment)
    result = {
        "url": url,
        "title": title,
        "metrics": {
            "word_count": word_count,
            "sentence_count": sentence_count,
            "flesch_reading_ease": round(flesch_reading_ease, 2)
        },
        "quality_assessment": {
            "predicted_label": quality_label,
            "probabilities": probabilities
        }
    }

    # Return it as a nicely formatted JSON-like string
    return json.dumps(result, indent=2)

# === Let's test it! ======================================================
# You can change this URL to any blog post
test_url = "https://www.varonis.com/blog/cybersecurity-tips" # (One from our dataset)
print(analyze_url(test_url))

test_url_2 = "https://neilpatel.com/blog/seo-copywriting/" # (A random one)
print(analyze_url(test_url_2))

Loaded trained model 'models/quality_model.joblib'

--- Analyzing https://www.varonis.com/blog/cybersecurity-tips ---
{
  "url": "https://www.varonis.com/blog/cybersecurity-tips",
  "title": "Top 10 Cybersecurity Awareness Tips: How to Stay Safe and Proactive",
  "metrics": {
    "word_count": 1700,
    "sentence_count": 92,
    "flesch_reading_ease": 41.47
  },
  "quality_assessment": {
    "predicted_label": "Medium",
    "probabilities": {
      "High": "31.7%",
      "Low": "12.2%",
      "Medium": "56.1%"
    }
  }
}

--- Analyzing https://neilpatel.com/blog/seo-copywriting/ ---
{'error': 'Failed to fetch. Status code: 403'}
